### Step 1 - Data Cleaning

Open all the recipes data in one file and prepare them for further analysis!

In [2]:
# Import the necessary library!
import pandas as pd
import os
import numpy as np
import glob
import datetime

import nltk
import re
import string
from nltk.tokenize import RegexpTokenizer  
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
print('package ready')

package ready


In [3]:
pd.set_option('display.max_rows', 100) # specifies number of rows to show
# specifies default number format to 4 decimal places
pd.options.display.float_format = '{:40,.2f}'.format 
pd.options.display.max_colwidth 
pd.options.display.max_colwidth = 500

In [4]:
# List all the files in the folder!  
files = glob.glob('guardian/*.csv')
# read the files in a dataframe  
data_df = [pd.read_csv(f) for f in files]

In [5]:
# Append all the files together  
data = pd.concat(data_df,ignore_index=True)
# Sort by date and Reset the index  
data = data.sort_values(['date']).reset_index()  
# Drop the duplicates  
data = data.drop_duplicates(['date','headline_list','author'])

In [6]:
print("Number of data (recipes) : " + str(data.shape))
print()
data.tail(2)

Number of data (recipes) : (7464, 11)



,index,Unnamed: 0,date,headline_list,category,author,subtitle,article_content,comments_number,share_number,url
7613,2263,65.00,META,NaN,NaN,"[<meta content=""the Guardian"" name=""author""/>]",NaN,No Content,No Comments,No Sharing,https://www.theguardian.com/lifeandstyle/video/2011/jan/16/miso-soup-recipe-video
7615,1613,160.00,META,NaN,NaN,"[<meta content=""Ruby Tandoh"" name=""author""/>]",NaN,No Content,No Comments,No Sharing,https://www.theguardian.com/lifeandstyle/ng-interactive/2015/oct/16/ruby-tandohs-butternut-squash-and-spinach-casserole-with-herbed-feta-crumble


In [7]:
# Drop the column that we don't need
data = data.drop(columns=['Unnamed: 0','index'])
data = data.drop_duplicates(['date','headline_list'],keep='first')

In [8]:
# Look for null value in the data
print(data.isnull().sum())

date                 0
headline_list        1
category             8
author             324
subtitle           187
article_content      1
comments_number      3
share_number         3
url                  2
dtype: int64


In [9]:
# Make a bit of cleaning
# Delete all the article with category 'Family Life' - Not relevant for recipes analysis! 
data = data[data['category'] != 'Family life']

In [ ]:
# Check at the name of the author
#for i in data.author:
  #  print(i)

In [10]:
# Replace the author that have been wrongly webscraped!  
data = data.replace({'[<meta content="Dale Berning Sawa" name="author"/>]':'Dale Berning Sawa',  
                     '[<meta content="Chi-chi Nwanoku" name="author"/>]':'Chi-chi Nwanoku',  
                     '[<meta content="Hugh Fearnley-Whittingstall" name="author"/>]':'Hugh Fearnley-Whittingstall',  
                     '[<meta content="Jeanette Winterson" name="author"/>]':'Jeanette Winterson',  
                     '[<meta content="Matthew Fort" name="author"/>]':'Matthew Fort',  
                     '[<meta content="Susan McCarthy" name="author"/>]':'Susan McCarthy',  
                     '[<meta content="Ariane Sherine" name="author"/>]':'Ariane Sherine',  
                     '[<meta content="Ruby Tandoh" name="author"/>]':'Ruby Tandoh',  
                     '[<meta content="the Guardian" name="author"/>]':'the Guardian',  
                     '[<meta content="Romy Ash" name="author"/>, <meta content="Sarah Trotter" name="author"/>, <meta content="Romy Ash" name="author"/>, <meta content="Lauren Bamford" name="author"/>, <meta content="Sarah Trotter" name="author"/>]':'Romy Ash, Sara Trotter, Lauren Bamford',  
                     '[<meta content="Clem Bastow" name="author"/>]':'Clem Bastow',  
                     '[<meta content="Yotam Ottolenghi" name="author"/>, <meta content="Uyen Luu" name="author"/>, <meta content="David Frenkiel" name="author"/>, <meta content="Luise Vindahl" name="author"/>, <meta content="Caroline Craig" name="author"/>]':'Caroline Craig, Luise Vindahl, Yotam Ottolenghi, Uyen Luu, David Frenkiel',  
                     'Yotam Ottlenghi':'Yotam Ottolenghi','Yuki Sugiura, Valerie Berry, Lee Gould, Rachel Vere and Music by Evan Gildersleeve, theguardian.com':'Yuki Sugiura, Valerie Berry, Lee Gould, Rachel Vere',
                     'Hugh-Fearnley Whittingstall':'Hugh Fearnley-Whittingstall','Hugh Fearnley-Whittinstall':'Hugh Fearnley-Whittingstall'})

# If NaN for authors and NaN for subtitle -> change it to none!   
values = {'author': 'none', 'subtitle':'none'}  
data = data.fillna(value=values)

In [11]:
# Check for the no comments and no url!
test = data[data['comments_number'].isnull()]  
test.share_number = 'No Sharing'  
data.append(test)

change = data[(data['date'] == '2015-01-20') & (data['author']== 'Ruby Tandoh' )]  
change.share_number = 52  
change.url = 'https://www.theguardian.com/lifeandstyle/2015/jan/20/best-baking-recipes-from-2014-ruby-tandoh'  
change.comments_number = 25  
data.append(change)

print("Change ok!")

Change ok!


/Users/audevuilliomenet/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [12]:
# Remove the rows if NaN is in url, headline_list, share_number and category after having make the above change!  
data = data.dropna(subset = ['headline_list','url','share_number','category'])  

print('Number of rows (recipes), data clean: ' + str(data.shape))
print("")
print('Data Clean')

Number of rows (recipes), data clean: (7390, 9)

Data Clean


In [ ]:
# Check the types of the data!
data.dtypes

In [13]:
data.head(2)

,date,headline_list,category,author,subtitle,article_content,comments_number,share_number,url
0,2008-09-10T00:01:00+0100,The G2 weekly recipe: Beetroot pickled eggs,The G2 weekly recipe,Allegra McEvedy,none,"B\noth pickling and beetroot are enjoying a renaissance at the moment, while eggs never go out of fashion. When I was over in Virginia, US, the other week my friend James, knowing I liked trying some of the more weird and wonderful local delicacies gave me one of these pink eggs that a friend of his had made.\nHe made his with the liquid from a store-bought jar of pickled beets, which was good and interesting, but a subsequent version made with fresh juice made them even more arresting and b...",No Comments,4,https://www.theguardian.com/lifeandstyle/2008/sep/10/eggs.recipe
1,2008-09-10T11:43:43+0100,Just peachy,Allotment blog,Claire Ptak,A report and recipe from the Slow Food Nation event in San Francisco,"W\nhen I picked my husband up at the San Francisco International airport on Sunday, it was with great trepidation that I admitted an infidelity. 'I have something to tell you,' I said, 'I've been cheating on you — with California.' His knowing smile beamed as he adjusted his sunglasses and put down the window of our borrowed Volvo.\n\nI took him straight to the Mission district, to our old favourite, La Taqueria, for the self-proclaimed ""best burritos and tacos in the world."" Full on beans, ...",0,0,https://www.theguardian.com/global/allotment/2008/sep/10/recipe


Change the data date to date instead of object!

In [14]:
#Separe the date in year and month
# Remove the strings of time in date. Keep only the day and the month.b  
data.date = data.date.str.replace('T\d.*','')  
data['date'] = pd.to_datetime(data['date'])

# Add a column in dataframe for the year, month and day  
data['year'] = data['date'].dt.year  
data['month'] = data['date'].dt.month_name()  
data['day'] = data['date'].dt.day 

In [ ]:
data_sub = data[0:2]
for j in data_sub['token_article_content']:
    print(j)
    print('')
    test = ' '.join([lemmatizer.lemmatize(w) for w in j])
    print(test)
    print('')
    data_sub['lem_test'] = test
    # data_sub['lem_test'] = data_sub['token_article_content'].apply(lambda x: ' '.join([lemmatizer.lemmatize(w) for w in j])
    
#data['lem_words'+str(i)] = ' '.join([lemmatizer.lemmatize(w) for w in data['token_'+str(i)]])
#data_sub['test_lem'] = ' '.join([lemmatizer.lemmatize(w) for w in data_sub['token_article_content']])
#data_sub['test_lem'] = data_sub['token_article_content'].apply(lambda x:' '.join(s for s in x))
#data_sub['test_lem_words'] = data_sub['test_lem'].apply(lambda x: lemmatizer.lemmatize(x))

In [ ]:
input_str='been had done languages cities mice'
lemmatizer.lemmatize('eggs')
data_sub['lem_test'] = data_sub['token_article_content'].apply(lambda t: ' '.join([lemmatizer.lemmatize(w) for w in t]))lemmatizer = WordNetLemmatizer()

In [15]:
## Function for tokenization and cleaning text!   
cleaning = ['headline_list','subtitle','article_content']  

stop = stopwords.words('english') 
stop_words = ['tsp','tbsp','min','max','youll','neednt','isnt','ive','hese','dont','prep']
stop_remove = stop + stop_words

lemmatizer = WordNetLemmatizer()

def cleandata(data):
        for i in cleaning:
                print(str(i))
                # Remove the punctuation  
                data['clean_'+str(i)] = data[i].apply(lambda x:x.translate(str.maketrans('','', string.punctuation)))
                print('step 1: punctuation remove')

                
                # Get all the words in lowercase  
                data['clean_'+str(i)] = data['clean_'+str(i)].str.lower()  
                # Remove the '\\n' line_brakes  
                data['clean_'+str(i)] = data['clean_'+str(i)].str.replace(r'\n\d\d\d|\n\d\d|\n\d|\n\½|\.\n',' ')
                data['clean_'+str(i)] = data['clean_'+str(i)].str.replace(r'\n',' ')
                # Remove the special character 'à,é,è'
                data['clean_'+str(i)] = data['clean_'+str(i)].str.replace(r'\ó','o')
                data['clean_'+str(i)] = data['clean_'+str(i)].str.replace(r'\ú','u')
                data['clean_'+str(i)] = data['clean_'+str(i)].str.replace(r'\é','e')
                data['clean_'+str(i)] = data['clean_'+str(i)].str.replace(r'\è','e')
                data['clean_'+str(i)] = data['clean_'+str(i)].str.replace(r'\á','a')
                data['clean_'+str(i)] = data['clean_'+str(i)].str.replace(r'\à','a')
                # Remove the special character and the number
                data['clean_'+str(i)] = data['clean_'+str(i)].str.replace(r"\’|\“|\”|\s\–|\'|\‘\…\ •",'')
                data['clean_'+str(i)] = data['clean_'+str(i)].str.replace(r'[0-9]+|\¼|\½|\¼½', '')
                
                # Special case for 'since_you_are_here'
                if i == 'article_content':
                    data['clean_'+str(i)] = data['clean_'+str(i)].str.replace(r'since you’re here.*','')
                print('step 2: words in lowercase')

                # Remove all the english stopwords!  
                data['nostop_words_'+'clean_'+str(i)] = data['clean_'+str(i)].apply(  
                lambda x: ' '.join([word for word in x.split() if word not in (stop_remove)])) 
                # Remove all the words that are shorter than 2 characters! (ml, g,)
                data['nostop_words_'+'clean_'+str(i)] = data['nostop_words_'+'clean_'+str(i)].str.split().map(
                lambda x: " ".join(s for s in x if len(s) > 2))
                
                # Special case for 'first word of article' - remove it due to the \n break cleaning!
                if i == 'article_content':
                    data['nostop_words_'+'clean_'+str(i)] = data['nostop_words_'+'clean_'+str(i)].str.partition(' ')[2]
                print('step 3: nonstopwords and special characters removed')

                # Tokenize the text  
                data['token_'+str(i)] = data['nostop_words_'+'clean_'+str(i)].apply(word_tokenize)
                print('step 4: words tokenize')
                
                # Lemmatize Words - Plural and Singular are treated the same!
                data['lemmatize_'+str(i)] = data['token_'+str(i)].apply(lambda t: ' '.join([lemmatizer.lemmatize(w) for w in t]))
                #data['lem_words'+str(i)] = ' '.join([lemmatizer.lemmatize(w) for w in data['token_'+str(i)]])
                print('step 5: words lemmatize')
                print('Data Clean Finish')
                print("")
        return

cleandata(data)

headline_list
step 1: punctuation remove
step 2: words in lowercase
step 3: nonstopwords and special characters removed
step 4: words tokenize
step 5: words lemmatize
Data Clean Finish

subtitle
step 1: punctuation remove
step 2: words in lowercase
step 3: nonstopwords and special characters removed
step 4: words tokenize
step 5: words lemmatize
Data Clean Finish

article_content
step 1: punctuation remove
step 2: words in lowercase
step 3: nonstopwords and special characters removed
step 4: words tokenize
step 5: words lemmatize
Data Clean Finish



In [16]:
data.head()

,date,headline_list,category,author,subtitle,article_content,comments_number,share_number,url,year,...,token_headline_list,lem_wordsheadline_list,clean_subtitle,nostop_words_clean_subtitle,token_subtitle,lem_wordssubtitle,clean_article_content,nostop_words_clean_article_content,token_article_content,lem_wordsarticle_content
0,2008-09-10,The G2 weekly recipe: Beetroot pickled eggs,The G2 weekly recipe,Allegra McEvedy,none,"B\noth pickling and beetroot are enjoying a renaissance at the moment, while eggs never go out of fashion. When I was over in Virginia, US, the other week my friend James, knowing I liked trying some of the more weird and wonderful local delicacies gave me one of these pink eggs that a friend of his had made.\nHe made his with the liquid from a store-bought jar of pickled beets, which was good and interesting, but a subsequent version made with fresh juice made them even more arresting and b...",No Comments,4,https://www.theguardian.com/lifeandstyle/2008/sep/10/eggs.recipe,2008,...,"[weekly, recipe, beetroot, pickled, eggs]",weekly recipe beetroot pickled egg,none,none,[none],none,b oth pickling and beetroot are enjoying a renaissance at the moment while eggs never go out of fashion when i was over in virginia us the other week my friend james knowing i liked trying some of the more weird and wonderful local delicacies gave me one of these pink eggs that a friend of his had made he made his with the liquid from a storebought jar of pickled beets which was good and interesting but a subsequent version made with fresh juice made them even more arresting and beautiful bo...,pickling beetroot enjoying renaissance moment eggs never fashion virginia week friend james knowing liked trying weird wonderful local delicacies gave one pink eggs friend made made liquid storebought jar pickled beets good interesting subsequent version made fresh juice made even arresting beautiful colour flavour dozen freerange eggs vinegar pickling distilled beetroot juice amount obtained juicing fresh raw beetroot golden granulated sugar boil pan water gently lower eggs using slotted sp...,"[pickling, beetroot, enjoying, renaissance, moment, eggs, never, fashion, virginia, week, friend, james, knowing, liked, trying, weird, wonderful, local, delicacies, gave, one, pink, eggs, friend, made, made, liquid, storebought, jar, pickled, beets, good, interesting, subsequent, version, made, fresh, juice, made, even, arresting, beautiful, colour, flavour, dozen, freerange, eggs, vinegar, pickling, distilled, beetroot, juice, amount, obtained, juicing, fresh, raw, beetroot, golden, granul...",pickling beetroot enjoying renaissance moment egg never fashion virginia week friend james knowing liked trying weird wonderful local delicacy gave one pink egg friend made made liquid storebought jar pickled beet good interesting subsequent version made fresh juice made even arresting beautiful colour flavour dozen freerange egg vinegar pickling distilled beetroot juice amount obtained juicing fresh raw beetroot golden granulated sugar boil pan water gently lower egg using slotted spoon sim...
1,2008-09-10,Just peachy,Allotment blog,Claire Ptak,A report and recipe from the Slow Food Nation event in San Francisco,"W\nhen I picked my husband up at the San Francisco International airport on Sunday, it was with great trepidation that I admitted an infidelity. 'I have something to tell you,' I said, 'I've been cheating on you — with California.' His knowing smile beamed as he adjusted his sunglasses and put down the window of our borrowed Volvo.\n\nI took him straight to the Mission district, to our old favourite, La Taqueria, for the self-proclaimed ""best burritos and tacos in the world."" Full on beans, ...",0,0,https://www.theguardian.com/global/allotment/2008/sep/10/recipe,2008,...,[peachy],peachy,a report and recipe from the slow food nation event in san francisco,report recipe slow food nation event san francisco,"[report, recipe, slow, food, nation, event,

In [18]:
data_clean = data[['date', 'headline_list', 'category', 'comments_number', 'share_number', 'url', 'year',
       'month', 'day', 'clean_headline_list', 'nostop_words_clean_headline_list', 'token_headline_list',
       'clean_subtitle', 'nostop_words_clean_subtitle', 'token_subtitle','clean_article_content', 
       'nostop_words_clean_article_content', 'token_article_content','lem_wordsheadline_list','lem_wordsheadline_list','lem_wordsarticle_content']]

In [19]:
# Save the Clean Text in a CSV file
data_clean.to_csv('guardian_clean_data_final.csv')
print('data clean and save in csv file')

data clean and save in csv file
